In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score
#from scipy.linalg import pinv2
from scipy import linalg
from sklearn.preprocessing import RobustScaler

In [3]:
data = pd.read_csv("iris_data.csv")
data= data.drop(['Unnamed: 0', 'Id'], axis=1)
X=data.drop(['Species'], axis=1)
y = data['Species']
X=X.values
y= y.values
y_2d = y[:, None]
y_one_hot = np.hstack([y_2d==i for i in range(3)]).astype(int)

# Normalize input data
X = RobustScaler().fit_transform(X)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

In [4]:
# Compute predictions on training and testing sets
def ELM_L2(l,  X_train, X_test, y_train, y_test,alpha):
    
    Betha=[]
    # Define parameters
    n = X_train.shape[1] #input_size
    k = y_train.shape[1] #output_size

    # Randomly initialize input-to-hidden weights, input_weights
    w = np.random.randn(n, l)

    # Compute hidden layer activations
    #H = np.tanh(X_train@w)
    H = np.maximum(np.dot(X_train, w), 0) # ReLU activation function
    
    # Compute output weights using Moore-Penrose pseudoinverse (with L2 regularization) (B=(H'H)M.H'y)
    B = np.dot(linalg.pinv(H.T.dot(H) + alpha * np.eye(l)), H.T.dot(y_train)) #output_weights

    train_preds = np.dot(H, B)
    
    test_H = np.dot(X_test, w)
    test_H = np.maximum(test_H, 0)  # ReLU activation function
    test_preds = np.dot(test_H, B)
    
    Betha=B
    # Convert predictions to class labels
    train_pred_labels = np.argmax(train_preds, axis=1)
    test_pred_labels = test_preds.argmax(axis=1)
    

    # Calculate accuracy
    train_accuracy = accuracy_score(np.argmax(y_train, axis=1), train_pred_labels)
    test_accuracy = accuracy_score(np.argmax(y_test, axis=1), test_pred_labels)
    
    
    return train_accuracy, test_accuracy, Betha, w
    

In [5]:
for alpha in [0.01,0.1, 1,10, 50, 100]:
    for l in [1, 10, 50, 60, 100]:
        train_accuracy, test_accuracy, B , W = ELM_L2(l, X_train, X_test, y_train, y_test,alpha)
        print(f"L2 alpha={alpha}, neurons = {l} & ELM training score: {train_accuracy:.4f}")
        print(f"Testing Accuracy: {test_accuracy:.4f}")
        print(f"Weight: {W}")
        #print(f"Betha: {B}")
        print()

L2 alpha=0.01, neurons = 1 & ELM training score: 0.5667
Testing Accuracy: 0.6000
Weight: [[-0.41550389]
 [-1.51144226]
 [ 0.15992002]
 [ 0.56736998]]

L2 alpha=0.01, neurons = 10 & ELM training score: 0.9500
Testing Accuracy: 0.9000
Weight: [[ 0.51485056 -0.82381927 -0.89458423 -1.92155145 -0.80791331 -0.17750467
   0.73828692  0.72969169  0.75196468 -0.20209329]
 [-0.78218473  0.1509983   0.34465781  0.14912974  0.23459579 -0.41144908
  -0.94799222  1.51848241 -0.14988021 -1.10935909]
 [-0.80310001 -0.92114207 -1.91109294 -0.39144493  0.1002251   0.02550938
  -0.09593297  1.99283072  1.89541944  1.47840338]
 [-0.04072921  0.93098122 -0.02877712 -0.83892782 -0.49692433  0.03991685
   0.25341011  0.67067667  0.64977346 -0.54071849]]

L2 alpha=0.01, neurons = 50 & ELM training score: 0.9583
Testing Accuracy: 0.9000
Weight: [[-0.55744686  0.57351473 -0.60789364 -2.27452496 -0.49383457 -0.3020176
  -0.19080312 -0.69228813  0.70200966 -0.21490385 -0.96220022 -0.71919039
  -2.40550415 -0.261